**Bert model op basis van Bert van Universiteit van Groningen. Context handling moet nog aangepast worden. Oversampling a.d.h.v de mediaan. dynamisch treshhold zoeken voor unknown.**

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

In [3]:
import pandas as pd
import re

# Drop unnecessary columns
columns_to_drop = ["TXT_file_name", "file_name", "statistical"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Handle missing values
df = df.dropna(subset=["question"])
df["context"].fillna("", inplace=True)



# Clean text function
def clean_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\b[a-z]\)\s*', ' ', text)
    text = re.sub(r'\b\d+\.\b', '', text)
    text = re.sub(r'\b\d+\)\b', '', text)
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d+[.)]\s*', '', text)
    text = re.sub(r'\b[ivxlcdm]+\s*[.)]\s*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'•', '', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Combine context and question, then clean
df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)

# Remove rows with the same clean_text but different themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme"].nunique().reset_index()
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]

# Map detailed themes to broader theme clusters
theme_merge_map = {
    # Bestuur en Beleid
    "Lokale overheden en Binnenlands bestuur": "Bestuur en Beleid",
    "Vlaamse administratie": "Bestuur en Beleid",
    "Staatshervorming en Verhoudingen binnen de Belgische federale staat": "Bestuur en Beleid",
    "Vlaamse Regering": "Bestuur en Beleid",
    "Vlaams Parlement": "Bestuur en Beleid",

    # Mobiliteit en Infrastructuur
    "Mobiliteit en Verkeer": "Mobiliteit en Infrastructuur",
    "Openbare werken": "Mobiliteit en Infrastructuur",
    "Ruimtelijke ordening": "Mobiliteit en Infrastructuur",

    # Economie en Arbeid
    "Werk": "Economie en Arbeid",
    "Economie": "Economie en Arbeid",
    "Sociale economie": "Economie en Arbeid",
    "Internationaal ondernemen": "Economie en Arbeid",

    # Welzijn en Gezondheid
    "Welzijn en Gezin": "Welzijn en Gezondheid",
    "Gezondheid": "Welzijn en Gezondheid",
    "Armoedebeleid": "Welzijn en Gezondheid",

    # Cultuur en Communicatie
    "Cultuur": "Cultuur en Communicatie",
    "Media": "Cultuur en Communicatie",
    "Taalgebruik": "Cultuur en Communicatie",

    # Onderwijs en Samenleving
    "Onderwijs en Vorming": "Onderwijs en Samenleving",
    "Gelijke kansen": "Onderwijs en Samenleving",
    "Jeugdbeleid": "Onderwijs en Samenleving",
    "Integratie en Inburgering": "Onderwijs en Samenleving",

    # Milieu en Landbouw
    "Natuur en Milieu": "Milieu en Landbouw",
    "Landbouw": "Milieu en Landbouw",
    "Dierenwelzijn": "Milieu en Landbouw",

    # Internationaal Beleid
    "Buitenlands beleid": "Internationaal Beleid",
    "Europese instellingen": "Internationaal Beleid",
    "Ontwikkelingssamenwerking": "Internationaal Beleid",
    "Oekraïnecrisis": "Internationaal Beleid",

    # Overige
    "Financiën": "Financiën",
    "Begroting": "Begroting",
    "Wetenschap en Innovatie": "Wetenschap en Innovatie",
    "Toerisme": "Toerisme",
    "Justitie en Handhaving": "Justitie en Handhaving",
    "Brussel en de Vlaamse Rand": "Brussel en de Vlaamse Rand",
    "Sport": "Sport",
    "Onroerend erfgoed": "Onroerend erfgoed",
    "Energie": "Energie",
    "Wonen": "Wonen",
}

# Apply theme mapping
df["theme"] = df["theme"].map(theme_merge_map).fillna("Onbekend")

# Remove rare themes
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme"].isin(valid_themes)]

# Recompute theme IDs
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

# Final row count
print(f"Number of rows after filtering: {len(df)}")


C:\Users\jefva\AppData\Local\Temp\ipykernel_5676\786054309.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)


Number of rows after filtering: 100059


In [4]:
print("All theme_ids:", sorted(df["theme_id"].unique()))
print("num_labels:", df["theme_id"].nunique())


All theme_ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
num_labels: 18


In [5]:
# ✅ 5. Split Data into Train & Test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["clean_text"].tolist(), df["theme_id"].tolist(), test_size=0.2, random_state=42, stratify=df["theme_id"]
)

In [6]:
from collections import Counter
import random

# Step 1: Identify the top class
theme_counts = Counter(train_labels)
top_class = max(theme_counts, key=theme_counts.get)
top_class_count = theme_counts[top_class]
print(f"🔻 Top class {top_class} has {top_class_count} samples")

# Step 2: Get all indices for that class
top_class_indices = [i for i, label in enumerate(train_labels) if label == top_class]

# Step 3: Randomly remove half
random.seed(42)
drop_count = len(top_class_indices) // 2
drop_indices = set(random.sample(top_class_indices, drop_count))

# Step 4: Filter train_texts and train_labels
train_texts = [text for i, text in enumerate(train_texts) if i not in drop_indices]
train_labels = [label for i, label in enumerate(train_labels) if i not in drop_indices]

print(f"✅ Removed {drop_count} samples from top class {top_class}")


🔻 Top class 9 has 21623 samples
✅ Removed 10811 samples from top class 9


In [7]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

# Compute class counts and use median as balancing target
theme_counts = train_df["theme_id"].value_counts()
median_count = theme_counts.median()

# Define strategy: only oversample underrepresented classes
sampling_strategy = {
    theme: int(median_count)
    for theme in theme_counts.index
    if theme_counts[theme] < median_count and theme_counts[theme] > 50 
}

# Apply RandomOverSampler
ros = RandomOverSampler(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = ros.fit_resample(train_df[["clean_text"]], train_df["theme_id"])

# Extract oversampled train lists
train_texts_resampled = X_resampled["clean_text"].tolist()
train_labels_resampled = y_resampled.tolist()

from collections import Counter
print("Class distribution after oversampling:", Counter(train_labels_resampled))


Class distribution after oversampling: Counter({2: 12382, 9: 10812, 13: 9076, 10: 8070, 4: 7086, 6: 5031, 8: 2854, 1: 2842, 12: 2646, 11: 2218, 3: 2218, 16: 2218, 14: 2218, 0: 2218, 5: 2218, 7: 2218, 15: 2218, 17: 2218})


C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [8]:
# ✅ 7. Load BERT Tokenizer & Define Dataset Class
model_name = "bert-base-multilingual-cased"  # mBERT
tokenizer = BertTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts_resampled, train_labels_resampled, tokenizer)
test_dataset = ThemeDataset(test_texts, test_labels, tokenizer)

In [9]:
# ✅ 8. Load BERT Model for Classification
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df["theme_id"].nunique())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from sklearn.utils.class_weight import compute_class_weight
import torch

# Assuming train_labels is a list of class indices
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels_resampled),
    y=train_labels_resampled
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

clipped_weights = torch.clamp(class_weights, min=0.5, max=10.0)#normalizing is ook een optie maar zitten hier met enorm skewed data dus clipping




In [11]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=clipped_weights.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [12]:
# ✅ 9. Define Training Arguments (With Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,                 # 👈 Keep only the last checkpoint
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ 11. Train Model with Early Stopping
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

trainer.train()

C:\Users\jefva\anaconda3\envs\bert_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.737800,0.431262,0.902558,0.905007,0.902558,0.902971
2,0.263000,0.355941,0.939986,0.941366,0.939986,0.940280
3,0.145200,0.290935,0.953328,0.954315,0.953328,0.953526
4,0.092300,0.264967,0.962772,0.963312,0.962772,0.962894
5,0.055100,0.265645,0.966020,0.966256,0.966020,0.966052


KeyboardInterrupt: 

In [13]:
trainer.save_model("output/mixed-bert-model")


In [14]:
# Get predictions from the model
preds_output = trainer.predict(test_dataset)

# Extract logits (raw model outputs)
logits = preds_output.predictions  # shape: (num_samples, num_classes)

# Get predicted class labels
predictions = np.argmax(logits, axis=1)

# Save everything in a DataFrame
output_df = pd.DataFrame({
    "text": test_texts,
    "true_label": test_labels,
    "predicted_label": predictions,
    "logits": logits.tolist()  # Save logits for softmax + threshold later
})

# Save to CSV
output_df.to_csv("output/test_predictions_mixed.csv", index=False)


In [15]:
# Final check before training
print("✔️ Final weights used in training:")
for i, weight in enumerate(class_weights):
    print(f"Class {i} → {weight.item():.4f}")


✔️ Final weights used in training:
Class 0 → 2.0229
Class 1 → 1.5787
Class 2 → 0.3624
Class 3 → 2.0229
Class 4 → 0.6332
Class 5 → 2.0229
Class 6 → 0.8918
Class 7 → 2.0229
Class 8 → 1.5721
Class 9 → 0.4150
Class 10 → 0.5560
Class 11 → 2.0229
Class 12 → 1.6957
Class 13 → 0.4944
Class 14 → 2.0229
Class 15 → 2.0229
Class 16 → 2.0229
Class 17 → 2.0229


In [16]:
# Final check before training
print("✔️ Final weights used in training:")
for i, weight in enumerate(clipped_weights):
    print(f"Class {i} → {weight.item():.4f}")

✔️ Final weights used in training:
Class 0 → 2.0229
Class 1 → 1.5787
Class 2 → 0.5000
Class 3 → 2.0229
Class 4 → 0.6332
Class 5 → 2.0229
Class 6 → 0.8918
Class 7 → 2.0229
Class 8 → 1.5721
Class 9 → 0.5000
Class 10 → 0.5560
Class 11 → 2.0229
Class 12 → 1.6957
Class 13 → 0.5000
Class 14 → 2.0229
Class 15 → 2.0229
Class 16 → 2.0229
Class 17 → 2.0229


SAVE MODEL NAAR KUL DRIVE 

In [16]:
from datetime import datetime

run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
#save_path = f"C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/mBERT/Run_{run_id}"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

NameError: name 'save_path' is not defined